# Fitting the damage function.

The purpose of this notebook is to create a continuous damage function based on the table specifying damage for different threshold classes. The output are a parameter file and some figures. 

In [ ]:
#%matplotlib notebook
%matplotlib inline

import os
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import minimize

DATADIR = "/home/erlend/data/portugal-rerun"
SRCDIR = "/home/erlend/github/gp-flood-damage-aggregator"
# Set environment variable DATADIR
%env DATADIR $DATADIR

#Local import (change working directory), set to project src dir.
os.chdir(SRCDIR)
os.getcwd()

from betapert import pert, plot_damage, check_stats
from config import DAMAGE_ROAD_STATIC, DAMAGE_ROAD_DYNAMIC

plt.rcParams['figure.figsize'] = [12,7]

In [ ]:
SCENARIOS = ["D312_APA_AI_T{}".format(ret) for ret in ["020", "100", "1000"]]

In [ ]:
# open features.tif for each scenario to get an overview of the raster values.
feature_file = os.path.join(DATADIR, "floodmaps/merged_floodmaps", "features.tif")

In [ ]:
feature_file

In [ ]:
with rasterio.open(feature_file) as dataset:
    print("Name: {}".format(dataset.name))
    depth = dataset.read(5).flatten()
    print("depth: {}".format(dataset.descriptions[4]))
    velocity = dataset.read(6).flatten()
    print("veloicty: {}".format(dataset.descriptions[5]))

In [ ]:
dataset.descriptions

Let us only consider depth in the range 0 to 6 meters. The dataset also contains fare larger depths, however these are usually where there are already rivers when there is no flooding, and hence not relevant for damage assessment. 

In [ ]:
depth_not_0 = np.logical_and(depth != 0, depth < 6)

In [ ]:
# Distribution of values.
plt.hist(depth[depth_not_0], bins = 50);

There is certainly too many samples at high depth range. One option is to rather sample values according to some chosen distribution. For now, we will keep it like this, assuming it does not have too large of an impact on the fitted values.

In [ ]:
np.sum(depth_not_0)

In [ ]:
plt.hist(velocity[depth_not_0], log=True, bins = 50);

## Correlation of velocity and depth.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.scatter(depth[depth_not_0][:50000], velocity[depth_not_0][:50000], alpha=0.1)
ax.set_xlabel("Depth")
ax.set_ylabel("Velocity")
plt.show()

## Fitting a suitable damage function.

The damage information is described in terms of a number from 0 to 1. The damage function relates depth, or depth and velocity to damage. The available information is given as three categories in terms of depth and two categories in terms of velocity. Cat1 (d<0.5), Cat2 (0.5 < d < 2.0), Cat3 (2<d) where d is depth. Further velocity is split in lower or higher than 1. Values of damage for each cathegory are expressed in a range of low, mean, high.

This provides means of sampling damage values as a function of depth and velocity. However, as seen below values are highly dependent on cathegory, in particular the velocity is highly discontinuous. Consequently a relatively simple model might reasonably well capture values, without the artificial discontinuity. Further it may provide more efficient sampling.

In [ ]:
plot_damage(DAMAGE_ROAD_DYNAMIC, samples=500000, bins=100, file_name="notebooks/figures/beta_dist.png")

In [ ]:
# Make a nicer plot of the distributions for article.

import seaborn as sns
from betapert import pert
sns.set(font_scale=1.2)
sns.set_style("whitegrid")

data = []
for nr, (key, (a, b, c)) in enumerate(DAMAGE_ROAD_DYNAMIC.items()):
    data.extend([(key, v) for v in pert(a, b, c, 500000)])

sampled_damage_df = pd.DataFrame(data, columns=["Category", "Damage"])

In [ ]:
ax = sns.displot(
    sampled_damage_df, 
    x="Damage", 
    hue="Category", 
    kind="kde", 
    fill=True, 
    bw_adjust= 1.5, 
    #aspect=1.8, 
    gridsize=200, 
    legend=False
)
ax.set(xlim=(0, 1.))
plt.legend(labels=["$2 < h$","$0.5 < h < 2$", "$h < 0.5$"])
plt.savefig("notebooks/figures/sns_beta_dist.png")

In [ ]:
DAMAGE_ROAD_DYNAMIC

In [ ]:
# To sample, simply call
pert(*DAMAGE_ROAD_DYNAMIC['Cat2'], 10)

In [ ]:
pd.DataFrame(DAMAGE_ROAD_STATIC, index=["minimum", "mode", "maximum"])

In [ ]:
pd.DataFrame(DAMAGE_ROAD_DYNAMIC, index=["minimum", "mode", "maximum"])

In [ ]:
def get_pert_parameters(depth, velocity):
    cat_nr = len([x for x in [0.5,2] if x < depth]) + 1
    if velocity < 1:
        return DAMAGE_ROAD_STATIC["Cat{}".format(cat_nr)]
    else:
        return DAMAGE_ROAD_DYNAMIC["Cat{}".format(cat_nr)]

In [ ]:
#df = pd.DataFrame({"depth" : np.random.uniform(low=0, high=5, size=1000),
#                            "velocity": np.random.uniform(low=0, high=2, size=1000)})
number_of_samples = 500000
df = pd.DataFrame({"depth" : depth[depth_not_0][:number_of_samples], 
                   "velocity": velocity[depth_not_0][:number_of_samples]})

In [ ]:
df

In [ ]:
df["damage"] = df.apply(lambda row: float(pert(*get_pert_parameters(row["depth"], row["velocity"]),1)[0]), axis=1)

In [ ]:
fig = df[:10000].plot.scatter("depth", "damage", alpha=0.3).get_figure()

In [ ]:
fig.savefig("notebooks/figures/depth-damage-treshold.png")

In [ ]:
df[:10000].plot.scatter("velocity", "damage", alpha=0.3)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
samples = 50000
ax.scatter(df[:samples]["depth"], df[:samples]["velocity"], df[:samples]["damage"], alpha=0.2)

It is clear that the threshold version is not ideal. Let us find a suitable substitute and fit the parameters. The recovered function will have to satisfy:

- Bounded in [0,1].
- monotone in both parameters.
- damage=0 for height=0 and velocity=0. 

One option is to transform the target according to 
$$
    d = \frac{L}{1 + L} \quad \Leftrightarrow \quad L = \frac{d}{1-d}.
$$
Then create a model so that $L = \hat L\exp(\varepsilon)$ and $\hat L$ is a model for $L$ based on a linear combination of $h, v$ and also possibly nonlinear terms $hv$ and $v^2$ as parameters. 

In [ ]:
df["l"] = df.apply(lambda row: row["damage"]/(1-row["damage"]), axis=1)

In [ ]:
# possible features.
df["velocity_sq"] = df.apply(lambda row: row["velocity"]**2, axis=1)
df["moment"] = df.apply(lambda row: row["velocity"]*row["depth"], axis=1)
df["const"] = df.apply(lambda row: 1., axis=1)

In [ ]:
df.dtypes

As we can tell from the above plot, $\varepsilon$ is close to normal, however as $\varepsilon$ is not centered $\hat L$ should be scaled in order to obtain a better fit for the damage values. That is, applying least square on the level of transformed values $L$ does not translate to an optimal fit for the damage values. This requires a nonlilnear optimization procedure. To this end note that 
$$
\varepsilon = \log(L/\hat L)
$$
Hence, a natural loss function $\sum \varepsilon_i^2 = \sum \log(L_i/\hat L_i)^2$. (Compare with cross entropy loss.)

In [ ]:
def l_hat(beta, depth, velocity):
    return np.abs(beta[0]*depth + beta[1]*velocity + beta[2]*depth*velocity**2)
    #return beta[0]*depth + beta[1]*velocity**2

def d_hat(beta, depth, velocity):
    l_preds = l_hat(beta, depth, velocity)
    return l_preds/(1 + l_preds)

def eps(beta, depth, velocity, l):
    return np.log(l) - np.log(l_hat(beta, depth, velocity))  
                              
def loss(beta):
    return np.mean(np.square(eps(beta, df["depth"], df["velocity"], df["l"])))

beta_0 = np.array([.01, .01, .01])
res = minimize(loss, beta_0, method='Nelder-Mead', tol=1e-12)

In [ ]:
res

Testing with different variables:

| d | dv | v | v^2 | dv^2 |loss |
|---|----|---|-----|------|-----|
| x | x  | x | x   |      |0.99 |
| x | x  | x |     |      |0.99 |
| x |    |   | x   |      |1.12 |
| x | x  |   |     |      |1.18 |
| x | x  |   | x   |      |1.11 |
| x |    | x |     |      |1.28 |
| x |    | x |     |  x   |0.97 |

It seems that a model usiong depth, velocity and kinetic energy is suitable. Hence, the prediction model looks like
$$
\hat d = \frac{\hat L}{1 + \hat L} \mbox{ where } \hat L = abs(\beta ^T x)
$$
where beta is the coefficients and $x = (d,v,dv^2)$

Let us consider the new residuals using the model derived from the nonlinear optimization.

In [ ]:
df["l_hat"] = l_hat(res.x, df["depth"], df["velocity"])
df["d_hat"] = d_hat(res.x, df["depth"], df["velocity"])
df["d_residuals"] = df["d_hat"] - df["damage"]
df["eps"] = eps(res.x, df["depth"], df["velocity"], df["l"])

In [ ]:
# Make nicer figure for article.

sns.set(font_scale=1.8)
sns.set_style("whitegrid")

ax = sns.histplot(df['eps'], kde=False, stat='density', bins=80)
ax.set(xlim=(-4.5, 4.5))
ax.set(xlabel='Residuals', ylabel='Density')

# Plot the PDF.
xmin, xmax = plt.xlim()
x_pdf = np.linspace(xmin, xmax, 100)
y_pdf = norm.pdf(x_pdf, df["eps"].mean(), df["eps"].std())
sns.lineplot(x=x_pdf,y=y_pdf, lw=2)

plt.savefig("notebooks/figures/sns-eps-density.png")

It appears to be quite close to normal!

In [ ]:
plt.hist(df["d_residuals"], bins=100);

In [ ]:
print("damage residuals mean: {}, std:{}".format(df["d_residuals"].mean(), df["d_residuals"].std()))

In [ ]:

df[0:20000].plot(kind="scatter", x="damage", y="eps", alpha=0.1, 
                 xlabel="predicted damage", ylabel="residuals", logx=True)
#plt.scatter(x=df[:20000]["d_hat"], y=df[:20000]["eps"], alpha=0.1)
plt.savefig("notebooks/figures/d_hat-eps.png")

In [ ]:
df[0:20000].plot(kind="scatter", x="d_hat", y="damage", alpha=0.1, 
                 xlabel="predicted damage", ylabel="damage", logx=True, logy=True)
#plt.scatter(x=df[:20000]["d_hat"], y=df[:20000]["eps"], alpha=0.1)
plt.savefig("notebooks/figures/d_hat-damage.png")

Recall that $\varepsilon$ is scaled with $l$. 

In [ ]:
# Nicer figure, for publication.

nr_of_samples=15000
sns.set(font_scale=1.3)
sns.set_style("whitegrid")

fig, axs = plt.subplots(1, 2, sharey=True)
threshold_size = 15
threshold_alpha = 0.5
blue = sns.color_palette()[0]
orange = sns.color_palette()[1]


df[0:nr_of_samples].plot(
    kind="scatter",
    x="depth", 
    y="damage", 
    color=orange, 
    alpha=threshold_alpha, 
    ax=axs[0], 
    label="thresholds",
    marker='s',
    s=threshold_size,
    edgecolors='black',
    linewidth=0.4
)
df[0:nr_of_samples].plot(
    kind="scatter",
    x="depth", 
    y="d_hat", 
    color=blue, 
    alpha=0.4, 
    label="fitted", 
    ax=axs[0],
    s=10,
    edgecolors='black',
    linewidth=0.3
)

df[0:nr_of_samples].plot(
    kind="scatter", 
    x="velocity", 
    y="damage", 
    color=orange, 
    alpha=threshold_alpha, 
    ax=axs[1], 
    s=threshold_size,
    marker='s',
    label="thresholds",
    edgecolors='black',
    linewidth=0.3
)
df[0:nr_of_samples].plot(
    kind="scatter", 
    x="velocity", 
    y="d_hat", 
    color=blue, 
    alpha=0.4, 
    label="fitted", 
    s=10,
    ax=axs[1],
    edgecolors='black',
    linewidth=0.4
)

axs[0].set_xlabel("Depth [m]")
axs[0].set_ylabel("Damage")
axs[1].set_xlabel("Velocity [m/s]")
for ax in axs:
    ax.get_legend().remove()
    ax.set_ylim(1e-4,1)
    ax.set_yscale("log")

fig.legend(
    labels=["thresholds", "fitted"], 
    loc="upper center", 
    bbox_to_anchor=(0.99, 0.87),
    fancybox=False, 
    shadow=False, 
    ncol=1)

fig.savefig("notebooks/figures/fitted-thresholds-scatter-vel-depth.png", bbox_inches = 'tight')

## sample damage

Lets sample values according to $L = \hat L \exp(\varepsilon)$. If we sample $\varepsilon$ according to the observed residuals we obtain a problem. The uncertainty generated by the fitted model will be larger than the one indicated in the table. This is due to the large discontinuities in the sampled model (which are the ones we want to get rid of). As a remedy one might like to scale $\varepsilon$ so as to obtain values that agrees well with the original bound in the damage table.

In [ ]:
eps_mean, eps_std = df["eps"].mean(), df["eps"].std()

In [ ]:
def sample_damage(depth, velocity, scale_factor):
    xi = np.random.normal(scale=scale_factor*eps_std, size=depth.size)
    l = l_hat(res.x, depth, velocity)*np.exp(xi)
    return l/(1+l)

In [ ]:
scale_factors = [0.3,0.4,0.5,1.0]
for scale_factor in scale_factors:
    df["d_sample_{}".format(scale_factor)] = sample_damage(df["depth"], df["velocity"], scale_factor)

In [ ]:
sns.set(font_scale=1.3)
sns.set_style("whitegrid")

nr_of_samples=15000
d_sample = "d_sample_1.0"

fig, axs = plt.subplots(1, 2, sharey=True)
threshold_size = 15
threshold_alpha = 0.5
blue = sns.color_palette()[0]
orange = sns.color_palette()[1]


df[0:nr_of_samples].plot(
    kind="scatter",
    x="depth", 
    y="damage", 
    color=orange, 
    alpha=threshold_alpha, 
    ax=axs[0], 
    label="thresholds",
    marker='s',
    s=threshold_size,
    edgecolors='black',
    linewidth=0.4
)
df[0:nr_of_samples].plot(
    kind="scatter",
    x="depth", 
    y=d_sample, 
    color=blue, 
    alpha=0.4, 
    label="sampled", 
    ax=axs[0],
    s=10,
    edgecolors='black',
    linewidth=0.3
)


df[0:nr_of_samples].plot(
    kind="scatter", 
    x="velocity", 
    y="damage", 
    color=orange, 
    alpha=threshold_alpha, 
    ax=axs[1], 
    s=threshold_size,
    marker='s',
    label="thresholds",
    edgecolors='black',
    linewidth=0.3
)
df[0:nr_of_samples].plot(
    kind="scatter", 
    x="velocity", 
    y=d_sample, 
    color=blue, 
    alpha=0.4, 
    label="sampled", 
    s=10,
    ax=axs[1],
    edgecolors='black',
    linewidth=0.4
)

axs[0].set_xlabel("Depth [m]")
axs[0].set_ylabel("Damage")
axs[1].set_xlabel("Velocity [m/s]")
for ax in axs:
    ax.get_legend().remove()
    ax.set_ylim(1e-4,1)
    ax.set_yscale("log")

fig.legend(
    labels=["thresholds", "sampled"], 
    loc="upper center", 
    bbox_to_anchor=(0.99, 0.87),
    fancybox=False,
    shadow=False, 
    ncol=1)

fig.savefig("notebooks/figures/sampled-thresholds-scatter-vel-depth-{}.png".format(d_sample), bbox_inches = 'tight')

In [ ]:
# Alternative figure.

import seaborn.objects as so
import matplotlib.colors as colors
#sns.set(font_scale=1.8)
sns.set_style("whitegrid")

plot_data_df = df[0:100000]
fig, axs = plt.subplots(1, 3, sharey=True, sharex=True)
#p1 = so.Plot(plot_data_df, "velocity", "depth", ax=ax) 
#p1.add(so.Dots(alpha=0.8), color = "damage")
norm = colors.LogNorm(1e-3,1)

p1 = sns.scatterplot(
    data=plot_data_df, 
    y="velocity", 
    x="depth", 
    hue="damage",
    hue_norm = norm,
    ax=axs[0],
    
)

p2 = sns.scatterplot(
    data=plot_data_df, 
    y="velocity", 
    x="depth", 
    hue="d_hat", 
    hue_norm = norm,
    ax=axs[1]
)

p2 = sns.scatterplot(
    data=plot_data_df, 
    y="velocity", 
    x="depth", 
    hue="d_sample_1.0",
    hue_norm = norm,
    ax=axs[2]
)
# Remove legends
for ax in axs:
    ax.get_legend().remove()
    ax.set_xlim(0,6)
    ax.set_ylim(1e-2,6)
    ax.set_yscale("log")

# Add colorbar

#Normalize(0,1)
cmap = sns.cubehelix_palette(light=1, as_cmap=True)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

#Add colorbar
left = axs[2].get_position().x1+0.05
bottom = axs[2].get_position().y0
width = 0.05
height = axs[2].get_position().height

cax = fig.add_axes([left,bottom,width,height])
fig.colorbar(sm, cax=cax, label="Damage")

plt.show()

## Recreating damage table from sampled values.

Is it possible to move in opposite direction, i.e. find estimates on the dynamic and static flooding for different ranges of depths using the fitted damage function?

In [ ]:
df["isCat3"] = (df["depth"] > 2)
df["isCat2"] = (df["depth"] < 2) & (df["depth"] > 0.5)
df["isCat1"] = (df["depth"] < 0.5)
df["isDyn"] = (df["velocity"] > 1.)
df["isStat"] = (df["velocity"] < 1.)

Lets first look at how the predicted values are distributed according to cathegory.

In [ ]:
from itertools import product
cat_stats_d_hat_df = pd.DataFrame(columns = ['cat', 'd', 'cont_mean', 'tresh_mean', 'samples'])
d_sample = "d_sample_1.0"

fig, axs = plt.subplots(6, figsize=(10,25))
i = 0
for cat,d in product(["isCat3","isCat2","isCat1"], ["isDyn", "isStat"]):
    subset = df[cat] & df[d]
    bins = np.linspace(0, max(df[subset]["damage"].max(), df[subset][d_sample].max()), 50)
    axs[i].hist(df[subset]["damage"], bins, alpha=0.5, label='thresholds')
    axs[i].hist(df[subset]["d_hat"], bins, alpha=0.5, label='continuous')
    axs[i].legend()
    axs[i].set_title("depth: {}, velocity: {}".format(
        cat.replace("is",""),
        d.replace("is","")
    ))
    cat_stats_d_hat_df = cat_stats_d_hat_df.append({'cat':cat, 'd': d, 'cont_mean': df[subset]["d_hat"].mean(), 
                         'tresh_mean': df[subset]["damage"].mean(), 'samples':df[subset].shape[0]}, ignore_index = True)
    i = i+1
fig.savefig("notebooks/figures/damage-table-hist-d_hat.png")

As one would expect, continuous predictions appears more smeared out in these cathgories. However, one should keep in mind that we have been picking cathegories conforming to the ones used for sampling. The continuous model is supposed to fit also those not conforming to the ones we have sampled from. Let us see what happens when we add noise to predicted values.

In [ ]:
cat_stats_s_df = pd.DataFrame(columns = ['cat', 'd', 'cont_mean', 'tresh_mean', 'samples'])
d_sample = "d_sample_1.0"

fig, axs = plt.subplots(6, figsize=(10,25))
i = 0
for cat,d in product(["isCat3","isCat2","isCat1"], ["isDyn", "isStat"]):
    subset = df[cat] & df[d]
    bins = np.linspace(0, max(df[subset]["damage"].max(), df[subset][d_sample].max()), 50)
    axs[i].hist(df[subset]["damage"], bins, alpha=0.5, label='thresholds')
    axs[i].hist(df[subset][d_sample], bins, alpha=0.5, label='continuous')
    axs[i].legend()
    axs[i].set_title("depth: {}, velocity: {}".format(
        cat.replace("is",""),
        d.replace("is","")
    ))
    cat_stats_s_df = cat_stats_s_df.append({'cat':cat, 'd': d, 'cont_mean': df[subset][d_sample].mean(), 
                         'tresh_mean': df[subset]["damage"].mean(), 'samples':df[subset].shape[0]}, ignore_index = True)
    i = i+1
fig.savefig("notebooks/figures/damage-table-hist-{}.png".format(d_sample))

In [ ]:
cat_stats_s_df

In [ ]:
cat_stats_d_hat_df

## Write fitted values to file.

In [ ]:
import json
from datetime import date

In [ ]:
list(res.x)

In [ ]:
float(d_sample[-3:])

In [ ]:
results = {
    "date": str(date.today()),
    "source": "damage-function.ipynb",
    "params": {"depth": res.x[0], "velocity": res.x[1], "depth_velocity_2": res.x[2]},
    "eps": {"mean": eps_mean, "std": eps_std},
    "d_sample": float(d_sample[-3:])
}

In [ ]:
with open('notebooks/damage-func-config.json', 'w') as outfile:
    json.dump(results, outfile)

## Unknown velocity.

In many cases velocity is unknown, and one needs a damage function that only depends on depth. Perhaps the most obvious way is to refit a function, only depending on depth, while still using the sampled values. This enables us to mix the two tables for dynamic and static flooding. 

In [ ]:
def l_hat(beta, depth):
    return np.abs(beta[0]*depth + beta[1]*depth**2)
    #return beta[0]*depth + beta[1]*velocity**2

def d_hat(beta, depth):
    l_preds = l_hat(beta, depth)
    return l_preds/(1 + l_preds)

def eps(beta, depth, l):
    return np.log(l) - np.log(l_hat(beta, depth))  
                              
def loss(beta):
    return np.mean(np.square(eps(beta, df["depth"], df["l"])))

beta_0 = np.array([.01, .01, .01])

#Refit the values!
res = minimize(loss, beta_0, method='Nelder-Mead', tol=1e-8)

In [ ]:
# Set new predictions
df["l_hat"] = l_hat(res.x, df["depth"])
df["d_hat"] = d_hat(res.x, df["depth"])
df["d_residuals"] = df["d_hat"] - df["damage"]
df["eps"] = eps(res.x, df["depth"], df["l"])

In [ ]:
# Histogram.
plt.hist(df["eps"], density=True, bins=60)

# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)

plt.plot(x, norm.pdf(x, df["eps"].mean(), df["eps"].std()))
plt.savefig("notebooks/figures/depth-eps-density.png")

In [ ]:
eps_mean = df["xi"].mean()
eps_std = df["xi"].std()
print("eps mean: {}, eps std:{}".format(xi_mean, xi_std))

In [ ]:
plt.hist(df["d_residuals"], bins=40);
plt.yscale('log')

In [ ]:
print("damage residuals mean: {}, std:{}".format(df["d_residuals"].mean(), df["d_residuals"].std()))

## Sampling

In [ ]:
def sample_damage(depth, scale_factor):
    xi = np.random.normal(scale=scale_factor*eps_std, size=depth.size)
    l = l_hat(res.x, depth)*np.exp(xi)
    return l/(1+l)

In [ ]:
scale_factors = [0.3,0.4,0.5,1.0]
for scale_factor in scale_factors:
    df["d_sample_{}".format(scale_factor)] = sample_damage(df["depth"], scale_factor)

In [ ]:
d_sample = "d_sample_1.0"
ax = df[:10000].plot(kind="scatter", x="depth", y=d_sample, color="blue", alpha=0.3, label="sampled")
fig = df[:10000].plot(kind="scatter", x="depth", y="damage", color="orange", alpha=0.3, ax=ax, label="thresholds").get_figure()
fig.savefig("notebooks/figures/depth-sampled-thresholds-scatter-depth-{}.png".format(d_sample))

In [ ]:
ax = df[:10000].plot(kind="scatter", x="velocity", y=d_sample, color="blue", alpha=0.3, label="sampled")
fig = df[:10000].plot(kind="scatter", x="velocity", y="damage", color="orange", alpha=0.3, ax=ax, label="thresholds").get_figure()
fig.savefig("notebooks/figures/depth-sampled-thresholds-scatter-velocity-{}.png".format(d_sample))

In [ ]:
from itertools import product

fig, axs = plt.subplots(3, figsize=(10,15))
i = 0
for cat in ["isCat3","isCat2","isCat1"]:
    subset = df[cat]
    bins = np.linspace(0, max(df[subset]["damage"].max(), df[subset][d_sample].max()), 50)
    axs[i].hist(df[subset]["damage"], bins, alpha=0.5, label='thresholds')
    axs[i].hist(df[subset][d_sample], bins, alpha=0.5, label='continuous')
    axs[i].legend()
    axs[i].set_title("depth: {}".format(cat.replace("is","")))
    i = i+1
    
fig.savefig("notebooks/figures/depth-damage-table-hist-{}.png".format(d_sample))

## Write fitted values to file.

In [ ]:
import json
from datetime import date

In [ ]:
list(res.x)

In [ ]:
float(d_sample[-3:])

In [ ]:
d_sample[-3:]

In [ ]:
results = {
    "date": str(date.today()),
    "source": "damage-function.ipynb", 
    "params": {"depth": res.x[0], "depth_2": res.x[1]},
    "epsilon": {"mean": eps_mean, "std": eps_std},
    "d_sample": float(d_sample[-3:]),
    "gamma": gamma
}

In [ ]:
with open('depth-damage-func-config.json', 'w') as outfile:
    json.dump(results, outfile)